In [ ]:
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include<iostream>
#include<stdlib.h>
#include "dcmtk/dcmimgle/dcmimage.h"
#include<algorithm>
#include<stdio.h>


#define TILE_WIDTH 16
#define TILE_HEIGHT 16

typedef  unsigned short int USint;

using namespace std;

__constant__ USint multiple2[16];

__constant__ char x[8];

__constant__ char y[8];


__global__ void getLBDP(USint *pdata,int h, int w, unsigned char *out) {

	int tx=blockIdx.x*blockDim.x+threadIdx.x;
	int ty=blockIdx.y*blockDim.y+threadIdx.y;

	//printf("inside ty=%d tx=%d  w=%d  h=%d",ty,tx,w,h);
	//printf("x[0]=%d",x[0]);


	__shared__ USint bpdata[TILE_HEIGHT+2][TILE_WIDTH+2];

	bpdata[threadIdx.y+1][threadIdx.x+1]=pdata[ty*w+tx];

	//printf("YESAYESUYSE");
	USint bitp[16];


	//Since notion of threadIdx.x is changed to threadIdx.y
	if(threadIdx.x==0)
	{
		if(tx != 0)
		{
			//printf("A");
			bpdata[threadIdx.y+1][0]=pdata[ty*w+tx-1];
			if(threadIdx.y==0 && ty!=0)
				bpdata[0][0]=pdata[(ty-1)*w+tx-1];
		}

	}
	if(threadIdx.x==TILE_WIDTH-1)
	{
		//printf("B");
		if(tx != w-1) {
			bpdata[threadIdx.y+1][threadIdx.x+2]=pdata[ty*w+tx+1];
			if(threadIdx.y==TILE_HEIGHT-1 && ty!=h-1)
				bpdata[TILE_HEIGHT+1][TILE_WIDTH+1]=pdata[(ty+1)*w+tx+1];
		}
	}

	if(threadIdx.y==0)
	{
		if(ty != 0)
		{
			//printf("C");
			bpdata[0][threadIdx.x+1]=pdata[(ty-1)*w+tx];
			if(threadIdx.x == TILE_WIDTH-1 && tx!=w-1)
				bpdata[0][TILE_WIDTH+1]=pdata[(ty-1)*w+tx+1];
		}
	}
	if(threadIdx.y==TILE_HEIGHT-1)
	{
		if(ty!=h-1)
		{
			//printf("D");
			bpdata[threadIdx.y+2][threadIdx.x+1]=pdata[(ty+1)*w+tx];
			if(threadIdx.x==0 && tx!=0)
				bpdata[TILE_HEIGHT+1][0]=pdata[(ty+1)*w+tx-1];
		}
	}



	/*if(threadIdx.y==0 && ty !=0) {
		bpdata[0][threadIdx.x]=pdata[(ty-1)*w+tx];
		if(threadIdx.x == TILE_WIDTH-1 && tx!=w-1)
			bpdata[0][TILE_WIDTH+1]=pdata[(ty-1)*w+tx+1];
	}
	if(threadIdx.y==TILE_HEIGHT-1 && ty != h-1)
	{
		bpdata[TILE_HEIGHT+1][threadIdx.x+1]=pdata[(ty+1)*w+tx];
		if(threadIdx.x == TILE_WIDTH-1 && tx !=0)
			bpdata[TILE_HEIGHT+1][0]=pdata[(ty+1)*w+tx-1];
	}
	if(threadIdx.x==0 && tx !=0)
	{
		bpdata[threadIdx.y+1][0]=pdata[ty*w+tx-1];
		if(threadIdx.y==0 && ty!=0)
			bpdata[0][0]=pdata[(ty-1)*w+tx-1];
	}
	if(threadIdx.x==TILE_WIDTH-1 && tx != w-1)
	{
		bpdata[threadIdx.y+1][TILE_WIDTH+1]=pdata[ty*w+tx+1];
		if(threadIdx.y == TILE_HEIGHT-1 && ty!=h-1)
			bpdata[TILE_HEIGHT+1][TILE_WIDTH+1]=pdata[(ty+1)*w+tx+1];
	}*/


	if(ty==0 || ty==h-1 || tx==0 || tx==w-1)
		return;

	__syncthreads();

	/*int m,n;
	if(ty==1 && tx==15)
	{
		for(m=0;m<16+2;m++)
		{
			for(n=0;n<16+2;n++)
				printf("%d   ",bpdata[m][n]);
			printf("\n");
		}
	}*/

	//char *out;
	//out=(char *)calloc((h-1)*(w-1),sizeof(char));

	//printf("LKJLKJLK");

	USint centre;
	centre=bpdata[threadIdx.y+1][threadIdx.x+1];
	//printf("CENTER=%d",centre);
	USint cur;
	USint pow;
	USint btot;
	//printf("BBBBBB");
	btot=0;

	//printf("before LOOP1");
	for(int l=0;l<16;l++)
		bitp[l]=0;
        for(int l=0;l<16;l++)
	{
		pow=1;

		//printf("cur = %d\n",cur);
		for(int k=0;k<8;k++)
		{
			cur=bpdata[threadIdx.y+1+y[k]][threadIdx.x+1+x[k]];
			//printf("cur = %d bitp[%d]=%d\n",cur,l,bitp[l]);
			if(cur & multiple2[l])
				bitp[l] = bitp[l] + pow;

			//printf("%d \n",pow);
			pow = pow*2;
			//printf("pow =%d\n",pow);
			//if(ty==1 && tx==15 && l==0)
			//	printf("%d ",cur);
		}
	}
	//printf("before LOOP2");
	for(int l=15;l>=0;l--)
	{
		//printf("bitp[%d]=%d\n",l,bitp[l]);
		if(bitp[l] > centre)
			btot = btot+multiple2[15-l];
	}
	//printf("xxxxx centre=%d btot=%d",centre,btot);
	out[(ty-1)*(w-2)+(tx-1)]=(unsigned char)(btot>>8);

	//if(ty==1 && tx==15)
	//	printf("%d ",out[(ty-1)*(w-2)+(tx-1)]);
}




int main(int argc, char *argv[]) {

	cudaEvent_t pstart,pstop;

	float elapsedTime;
	cudaEventCreate(&pstart);
	cudaEventCreate(&pstop);

	cudaEventRecord(pstart,0);

	DicomImage *img=new DicomImage("../images/512x512CT.dcm");

	if(img != NULL && img->getStatus()==EIS_Normal) {
		if(img->isMonochrome()) {
			img->setMinMaxWindow();

			int h=img->getHeight();
			int w=img->getWidth();

			/*int h=3;
                        int w=3;

                        USint pdata[]={10,99,70,1,25,82,55,71,35};
			correct output = 238

				*/

			//cout<<"size = "<<img->getOutputDataSize(8)<<" "<<sizeof(Uint)<<" ";
			USint *pdata = (USint *)new USint[h*w];

			//cout<<"  "<<sizeof(pdata)<<" ";
			//(Uint *)img->getOutputData(8);

			//cout<<"Depth = "<<img->getDepth();

			img->getOutputData(pdata,w*h*sizeof(USint));


			USint *d_pdata;
			cudaMalloc((void **)&d_pdata,w*h*sizeof(USint));

			cudaMemcpy(d_pdata,pdata,w*h*sizeof(USint),cudaMemcpyHostToDevice);

			//double min;
			//double max;
			//img->getMinMaxValues(min,max);

			//cout<<"Min = "<<min<<" max = "<<max;

			//DiPixel *indata = (DiPixel *)img->getInterData();

			//for(int i=0;i<8;i++)
			//	cout<<pdata[i]<<"  ";

			/*if(pdata != NULL) {

				//cout<<w<<"   "<<h;

				for(int i=0;i<h;i++) {
					for(int j=0;j<w;j++) {
						if(pdata[i*w+j])
							cout<<pdata[i*w+j]<<" ";
					}
					//cout<<endl;
				}
			*/
			/*ofstream ofs;
			ofs.open("MR.pgm");
			ofs<<"P2"<<endl<<w<<endl<<h<<endl;
			ofs<<65535<<endl;

			if(pdata != NULL) {

				//cout<<w<<"   "<<h;

				for(int i=0;i<h;i++) {
					for(int j=0;j</*int h=3;
                        int w=3;*/

                        //USint pdata[]={15,0,56,0,50,0,160,0,100};w;j++) {
			/*			ofs<<pdata[i*w+j]<<endl;
					}
				}
			}*/


                     	unsigned char *out;

			out=(unsigned char *)calloc((h-2)*(w-2),sizeof(unsigned char));

			unsigned char *d_out;
			cudaMalloc((void **)&d_out,((h-2)*(w-2)*sizeof(unsigned char)));

			USint *h_multiple2;
			h_multiple2=(USint *)calloc(sizeof(USint)*8,sizeof(USint));

			h_multiple2[0]=1;
			for(int l=1;l<16;l++)
			{
				h_multiple2[l]=h_multiple2[l-1]<<1;
				//printf("%d   \n",multiple2[l]);
			}
			char h_x[8]={1,1,0,-1,-1,-1,0,1};
			char h_y[8]={0,-1,-1,-1,0,1,1,1};

			cudaMemcpyToSymbol(multiple2,h_multiple2,sizeof(USint)*8*sizeof(USint),0,cudaMemcpyHostToDevice);

			cudaMemcpyToSymbol(x,h_x,sizeof(char)*8,0,cudaMemcpyHostToDevice);
			cudaMemcpyToSymbol(y,h_y,sizeof(char)*8,0,cudaMemcpyHostToDevice);


			cudaEvent_t kstart, kstop;

			dim3 blk(std::max(w/TILE_WIDTH,1),std::max(h/TILE_HEIGHT,1));
			dim3 thrd(std::min(w,TILE_WIDTH),std::min(h,TILE_HEIGHT));
			//dim3 thrd(3,3);

			cudaEventCreate(&kstart);
			cudaEventCreate(&kstop);

			cudaEventRecord(kstart, 0);
			getLBDP<<<blk,thrd>>>(d_pdata,h,w,d_out);

			cudaEventRecord(kstop, 0);
			cudaEventSynchronize(kstop);
			float elapsedTime;
			cudaEventElapsedTime(&elapsedTime, kstart, kstop);

			printf("Kernel Elapsed time =%f\n",elapsedTime);

			cudaMemcpy(out,d_out,((h-2)*(w-2)*sizeof(unsigned char)),cudaMemcpyDeviceToHost);

			/*
			for(int i=1;i<h-1;i++)
			{
				for(int j=1;j<w-2;j++)
				{
					/*
					unsigned char  ndig=0;
					unsigned char pdig=1;
					unsigned char btot=0;
					for(int k=-1;k<2;k++)
					{
						for(int l=-1;l<2;l++)
						{
								if(!(k|l))
									continue;
								if(pdata[i*w+j]>pdata[(i+k)*w+(j+l)])
									btot+=pdig;
								ndig++;
								pdig*=2;
						}
					}
					if(btot !=out[(i-1)*(w-1)+(j-1)])
						printf("Error i=%d,j=%d btot=%d out=%d\n",i,j,btot,out[(i-1)*(w-1)+(j-1)]);
					*/
			/*		cout<<"h="<<i<<"  w="<<j<<"  "<<((USint)out[(i-1)*(w-2)+(j-1)])<<endl;
				}
				cout<<endl;
					;
			}*/

			//cout<<sizeof(pdata);
			//cout<<" "<<sizeof(indata);
			//ofs.close();
			delete out;
			//delete pdata;
			//delete indata;
		}
	}

	cudaEventRecord(pstop, 0);
	cudaEventSynchronize(pstop);
	cudaEventElapsedTime(&elapsedTime, pstart, pstop);
	printf("Program Elapsed time =%f\n",elapsedTime);

	return 0;
}


In [11]:
!sudo apt update
!sudo apt install libopencv-dev python3-opencv

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [783 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,635 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://security.ubuntu.com/ubunt

In [56]:
%%writefile main1.cu

#include<iostream>

#include <opencv4/opencv2/core.hpp>
#include <opencv4/opencv2/highgui.hpp>
#include <opencv4/opencv2/opencv.hpp>

#include <math.h>
#include <stdlib.h>
#include <chrono>

#include "cuda_runtime.h"

using namespace cv;
using namespace std;
using namespace std::chrono;

#define IMAGE_FILE "./image.jpg"

#define TILE_WIDTH 16
#define TILE_HEIGHT 16

typedef  unsigned short int USint;

using namespace std;

__constant__ int multiple2[16];

__constant__ char x[8];

__constant__ char y[8];

__device__ int binToDec1(int num) {
    int decimal_num = 0, base = 1, rem;

    while ( num > 0) {
        rem = num % 10;
        decimal_num = decimal_num + rem * base;
        num = num / 10;
        base = base * 2;
    }

    return decimal_num;
}

__device__ void decToBin(int num, int *arr) {
    int i=2;
    while(num > 0) {
        arr[i] = num % 2;
        num = num / 2;
        i--;
    }
}

__device__ int boundaryTraversal_signedArr(int arr[3][3], int N, int M) {
    int binVal = 0;
    for (int i = 0; i < M; i++) {
        binVal = binVal * 10 + arr[0][i];
    }

    for (int i = 1; i < N; i++) {
        binVal = binVal * 10 + arr[i][M-1];
    }

    if (N > 1) {
        for (int i = M - 2; i >= 0; i--) {
            binVal = binVal * 10 + arr[N - 1][i];
        }
    }

    if (M > 1) {
        for (int i = N - 2; i > 0; i--) {
            binVal = binVal * 10 + arr[i][0];
        }
    }

    return binToDec1(binVal);
}

__device__ void boundaryTraversal_magnitudeArr(int arr[3][3], int N, int M, int *newArr) {
    int pos=0;
    //converting 2D array to 1D array in clockwise manner
    for (int i = 0; i < M; i++) {
        newArr[pos] = arr[0][i];
        pos++;
    }
    for (int i = 1; i < N; i++) {
        newArr[pos] = arr[i][M-1];
        pos++;
    }
    if (N > 1) {
        for (int i = M - 2; i >= 0; i--) {
            newArr[pos] = arr[N - 1][i];
            pos++;
        }
    }
    if (M > 1) {
        for (int i = N - 2; i > 0; i--) {
            newArr[pos] = arr[i][0];
            pos++;
        }
    }
}

__device__ void lbp_3d_calculation(int arr1[3][3], int midVal, int *arr2) {
    int magnitudeArr[3][3], signArr[3][3], newArr[8];
    for(int i=0; i<3; i++) {
        for(int j=0; j<3; j++) {
            if(i==1 && j==1) {
                magnitudeArr[i][j] = 0;
                signArr[i][j] = 0;
            } else {
                int val = arr1[i][j] - midVal;
                if(val > 0) {
                    magnitudeArr[i][j] = val;
                    signArr[i][j] = 1;
                } else {
                    magnitudeArr[i][j] = -val;
                    signArr[i][j] = 0;
                }
            }
        }
    }

    arr2[0] = boundaryTraversal_signedArr(signArr, 3, 3);
    boundaryTraversal_magnitudeArr(magnitudeArr, 3, 3, newArr);

    int a1Val=0, a2Val=0, a3Val=0;
    for(int i=0; i<8; i++) {
        int binArr[3] = {0, 0, 0};
        newArr[i] > 7 ? decToBin(7, binArr) : decToBin(newArr[i], binArr);

        a1Val = a1Val * 10 + binArr[0];
        a2Val = a2Val * 10 + binArr[1];
        a3Val = a3Val * 10 + binArr[2];
    }

    arr2[1] = binToDec1(a1Val);
    arr2[2] = binToDec1(a2Val);
    arr2[3] = binToDec1(a3Val);

}

__global__ void mainKernal(int *pdata, int h, int w, int *layer1, int *layer2, int *layer3, int *layer4) {

  int tx=blockIdx.x*blockDim.x+threadIdx.x;
	int ty=blockIdx.y*blockDim.y+threadIdx.y;

	//printf("inside ty=%d tx=%d  w=%d  h=%d",ty,tx,w,h);
	//printf("x[0]=%d",x[0]);


	__shared__ int bpdata[TILE_HEIGHT+2][TILE_WIDTH+2];

	bpdata[threadIdx.y+1][threadIdx.x+1]=pdata[ty*w+tx];

	//printf("YESAYESUYSE");
	int bitp[16];


	//Since notion of threadIdx.x is changed to threadIdx.y
	if(threadIdx.x==0)
	{
		if(tx != 0)
		{
			//printf("A");
			bpdata[threadIdx.y+1][0]=pdata[ty*w+tx-1];
			if(threadIdx.y==0 && ty!=0)
				bpdata[0][0]=pdata[(ty-1)*w+tx-1];
		}

	}
	if(threadIdx.x==TILE_WIDTH-1)
	{
		//printf("B");
		if(tx != w-1) {
			bpdata[threadIdx.y+1][threadIdx.x+2]=pdata[ty*w+tx+1];
			if(threadIdx.y==TILE_HEIGHT-1 && ty!=h-1)
				bpdata[TILE_HEIGHT+1][TILE_WIDTH+1]=pdata[(ty+1)*w+tx+1];
		}
	}

	if(threadIdx.y==0)
	{
		if(ty != 0)
		{
			//printf("C");
			bpdata[0][threadIdx.x+1]=pdata[(ty-1)*w+tx];
			if(threadIdx.x == TILE_WIDTH-1 && tx!=w-1)
				bpdata[0][TILE_WIDTH+1]=pdata[(ty-1)*w+tx+1];
		}
	}
	if(threadIdx.y==TILE_HEIGHT-1)
	{
		if(ty!=h-1)
		{
			//printf("D");
			bpdata[threadIdx.y+2][threadIdx.x+1]=pdata[(ty+1)*w+tx];
			if(threadIdx.x==0 && tx!=0)
				bpdata[TILE_HEIGHT+1][0]=pdata[(ty+1)*w+tx-1];
		}
	}



	/*if(threadIdx.y==0 && ty !=0) {
		bpdata[0][threadIdx.x]=pdata[(ty-1)*w+tx];
		if(threadIdx.x == TILE_WIDTH-1 && tx!=w-1)
			bpdata[0][TILE_WIDTH+1]=pdata[(ty-1)*w+tx+1];
	}
	if(threadIdx.y==TILE_HEIGHT-1 && ty != h-1)
	{
		bpdata[TILE_HEIGHT+1][threadIdx.x+1]=pdata[(ty+1)*w+tx];
		if(threadIdx.x == TILE_WIDTH-1 && tx !=0)
			bpdata[TILE_HEIGHT+1][0]=pdata[(ty+1)*w+tx-1];
	}
	if(threadIdx.x==0 && tx !=0)
	{
		bpdata[threadIdx.y+1][0]=pdata[ty*w+tx-1];
		if(threadIdx.y==0 && ty!=0)
			bpdata[0][0]=pdata[(ty-1)*w+tx-1];
	}
	if(threadIdx.x==TILE_WIDTH-1 && tx != w-1)
	{
		bpdata[threadIdx.y+1][TILE_WIDTH+1]=pdata[ty*w+tx+1];
		if(threadIdx.y == TILE_HEIGHT-1 && ty!=h-1)
			bpdata[TILE_HEIGHT+1][TILE_WIDTH+1]=pdata[(ty+1)*w+tx+1];
	}*/


	if(ty==0 || ty==h-1 || tx==0 || tx==w-1)
		return;

	__syncthreads();

  //tx -> col, ty -> row
  int row = ty, col = tx;

  int idx = row * w + col;

    if(ty < h-1 && tx < w-1) {
        int a1[3][3], a2[4];
        for(int i=row, i1=0; i<row+3; i++, i1++) {
            for(int j=col, j1=0; j<col+3; j++, j1++) {
                a1[i1][j1] = (int)bpdata[i * w + j];
            }
        }

        int midVal = a1[1][1];

        lbp_3d_calculation(a1, midVal, a2);

        int magnitudeArr[3][3], signArr[3][3], newArr[8];
        for(int i=0; i<3; i++) {
            for(int j=0; j<3; j++) {
                if(i==1 && j==1) {
                    magnitudeArr[i][j] = 0;
                    signArr[i][j] = 0;
                } else {
                    int val = a1[i][j] - midVal;
                    if(val > 0) {
                        magnitudeArr[i][j] = val;
                        signArr[i][j] = 1;
                    } else {
                        magnitudeArr[i][j] = -val;
                        signArr[i][j] = 0;
                    }
                }
            }
        }

        layer1[idx] = boundaryTraversal_signedArr(signArr, 3, 3);

        boundaryTraversal_magnitudeArr(magnitudeArr, 3, 3, newArr);

        int a1Val=0, a2Val=0, a3Val=0;
        for(int i=0; i<8; i++) {
            int binArr[3] = {0, 0, 0};
            newArr[i] > 7 ? decToBin(7, binArr) : decToBin(newArr[i], binArr);

            a1Val = a1Val * 10 + binArr[0];
            a2Val = a2Val * 10 + binArr[1];
            a3Val = a3Val * 10 + binArr[2];
        }

        layer2[idx] = binToDec1(a1Val);
        layer3[idx] = binToDec1(a2Val);
        layer4[idx] = binToDec1(a3Val);
    }
}

int main(int argc, char *argv[]) {

    auto startTime = high_resolution_clock::now();

    cudaEvent_t start, end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    Mat image = imread(IMAGE_FILE, IMREAD_GRAYSCALE);
    int W = image.cols, H = image.rows;
    int size = W * H * sizeof(int);

    int imgArr[H * W], layer1[H * W], layer2[H * W], layer3[H * W], layer4[H * W];
    for(int i=0; i<H; i++) {
        for(int j=0; j<W; j++) {
            imgArr[i * W + j] = (int)image.at<uchar>(i, j);
        }
    }

    Mat result_image_layer1(H, W, CV_8UC1);
    Mat result_image_layer2(H, W, CV_8UC1);
    Mat result_image_layer3(H, W, CV_8UC1);
    Mat result_image_layer4(H, W, CV_8UC1);

    // Creating the 4 Layers
    int *d_imgArr, *d_layer1, *d_layer2, *d_layer3, *d_layer4;

    cudaMalloc((void**)&d_imgArr, size);
    cudaMalloc((void**)&d_layer1, size);
    cudaMalloc((void**)&d_layer2, size);
    cudaMalloc((void**)&d_layer3, size);
    cudaMalloc((void**)&d_layer4, size);

    cudaMemcpy(d_imgArr, imgArr, size, cudaMemcpyHostToDevice);

   //dim3 blockDim(16, 16);
    //dim3 gridDim((W + blockDim.x - 1) / blockDim.x, (H + blockDim.y - 1) / blockDim.y);

    dim3 blk(std::max(W/TILE_WIDTH,1),std::max(H/TILE_HEIGHT,1));
		dim3 thrd(std::min(W,TILE_WIDTH),std::min(H,TILE_HEIGHT));

    cudaEventRecord(start, 0);

    //mainKernal<<<gridDim, blockDim>>>(d_imgArr, H, W, d_layer1, d_layer2, d_layer3, d_layer4);
    mainKernal<<<blk,thrd>>>(d_imgArr, H, W, d_layer1, d_layer2, d_layer3, d_layer4);

    cudaEventRecord(end, 0);

    cudaMemcpy(layer1, d_layer1, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(layer2, d_layer2, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(layer3, d_layer3, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(layer4, d_layer4, size, cudaMemcpyDeviceToHost);

    float elapsed_time_ms;
    cudaEventElapsedTime(&elapsed_time_ms, start, end);
    printf("Kernel Execution Time: %.10fms\n", elapsed_time_ms);

/*
    for(int i=0; i<H; i++) {
      for(int j=0; j<W; j++) {
        result_image_layer1.at<uchar>(i, j) = static_cast<uchar>(layer1[i * W + j]);
        result_image_layer2.at<uchar>(i, j) = static_cast<uchar>(layer2[i * W + j]);
        result_image_layer3.at<uchar>(i, j) = static_cast<uchar>(layer3[i * W + j]);
        result_image_layer4.at<uchar>(i, j) = static_cast<uchar>(layer4[i * W + j]);
      }
    }

    imwrite("./output1/Layer1-Img.jpg", result_image_layer1);
    imwrite("./output1/Layer2-Img.jpg", result_image_layer2);
    imwrite("./output1/Layer3-Img.jpg", result_image_layer3);
    imwrite("./output1/Layer4-Img.jpg", result_image_layer4);
*/
    auto endTime = high_resolution_clock::now();

    cout<<"\nTime Taken by entire 3D-LBP program: "<<duration_cast<milliseconds>(endTime - startTime).count()<<"ms"<<endl;

    // Free device memory
    cudaFree(d_imgArr);
    cudaFree(d_layer1);
    cudaFree(d_layer2);
    cudaFree(d_layer3);
    cudaFree(d_layer4);

    return 0;
}

Overwriting main1.cu


In [57]:
!nvcc -o main1 main1.cu `pkg-config --cflags --libs opencv4`
!./main1

main1.cu(245): warning #767-D: conversion from pointer to smaller integer
                  a1[i1][j1] = (int)bpdata[i * w + j];
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

main1.cu(245): warning #767-D: conversion from pointer to smaller integer
                  a1[i1][j1] = (int)bpdata[i * w + j];
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

main1.cu(158): warning #177-D: variable "bitp" was declared but never referenced
   int bitp[16];
       ^

Kernel Execution Time: 0.3557760119ms

Time Taken by entire 3D-LBP program: 126ms


In [58]:
!nvcc -o main1 main1.cu `pkg-config --cflags --libs opencv4`
!./main1

main1.cu(245): warning #767-D: conversion from pointer to smaller integer
                  a1[i1][j1] = (int)bpdata[i * w + j];
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

main1.cu(245): warning #767-D: conversion from pointer to smaller integer
                  a1[i1][j1] = (int)bpdata[i * w + j];
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

main1.cu(158): warning #177-D: variable "bitp" was declared but never referenced
   int bitp[16];
       ^

Kernel Execution Time: 0.3071359992ms

Time Taken by entire 3D-LBP program: 127ms


In [59]:
!nvcc -o main1 main1.cu `pkg-config --cflags --libs opencv4`
!./main1

main1.cu(245): warning #767-D: conversion from pointer to smaller integer
                  a1[i1][j1] = (int)bpdata[i * w + j];
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

main1.cu(245): warning #767-D: conversion from pointer to smaller integer
                  a1[i1][j1] = (int)bpdata[i * w + j];
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

main1.cu(158): warning #177-D: variable "bitp" was declared but never referenced
   int bitp[16];
       ^

Kernel Execution Time: 0.3014079928ms

Time Taken by entire 3D-LBP program: 123ms
